### SMILES 로부터 descriptor 및 graph 특징 추출

In [3]:
import sys
import torch
import pandas as pd
from tqdm import tqdm
from torch_geometric.data import Data, Batch

sys.path.append("../src/")
from preprocess import get_descriptors_2d, get_descriptors_3d, get_ecfp

In [2]:
df = pd.read_pickle("../data/250714_preprocessed_HW_V2.pkl")
df.columns

Index(['smiles', 'ic50_nm', 'ic50_nm_imputed', 'is_active(10um)',
       'data_source', 'data_id', 'pvalue', 'pvalue_imputed', 'rdmol',
       'rdmol_confs5', 'medai_id'],
      dtype='object')

In [4]:
save2dir = "../data/250716_preprocess_hw"

# 컬럼 인덱스 계산
smiles_idx = df.columns.get_loc('smiles')
pvalue_idx = df.columns.get_loc('pvalue')
pvalue_imputed_idx = df.columns.get_loc('pvalue_imputed')
rdmol_idx = df.columns.get_loc('rdmol_confs5')
mol_id_idx = df.columns.get_loc('medai_id')
is_active_idx = df.columns.get_loc('is_active(10um)')

for i in tqdm(range(df.shape[0])):
    smiles = df.iloc[i, smiles_idx]
    pvalue = df.iloc[i, pvalue_idx]
    pvalue_imputed = df.iloc[i, pvalue_imputed_idx]
    rd_mol = df.iloc[i, rdmol_idx]
    mol_id = df.iloc[i, mol_id_idx]
    is_active = df.iloc[i, is_active_idx]
    
    # Feature 계산
    desc_2d = get_descriptors_2d(rd_mol)
    desc_3d = get_descriptors_3d(rd_mol)
    ecfp = get_ecfp(rd_mol, radius=2, fpSize=2048)
    
    # pyg Data 객체 형태로 만들기
    data = Data()
    data.desc_2d = desc_2d
    data.desc_3d = desc_3d
    data.ecfp = ecfp
    
    # Graph feature 추가
    
    # 저장
    torch.save(data, f"{save2dir}/{mol_id}.pt")

100%|██████████| 25753/25753 [04:52<00:00, 88.06it/s] 
